In [183]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import distance
import re

from cleaning_utils import * # import all existing functions from file cleaning.py

%load_ext autoreload
%autoreload 2


pd.set_option('display.max_columns', 360)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
blight = pd.read_csv('data/Blight.csv')
mcm = pd.read_csv('data/MCM.csv')
crimes = pd.read_csv('data/Crimes.csv')

In [186]:
df = pd.merge(blight,mcm,left_on='parcel_id',right_on='parcel_id',how='left')

In [187]:
df.columns

Index(['lon', 'lat', 'ticket_id', 'violation_street_number',
       'violation_street_name', 'violation_zip_code', 'violation_date',
       'judgment_date', 'payment_date', 'payment_status', 'parcel_id',
       'compliance', 'coordinates', 'AddCombo', 'Structure', 'Condition',
       'Occupancy', 'Needs_Boarding', 'Fire', 'Dumping', 'Units', 'Improved',
       'Maintained'],
      dtype='object')

In [188]:
df = df.dropna(subset=['AddCombo', 'Structure', 'Condition',
       'Occupancy', 'Needs_Boarding', 'Fire', 'Dumping', 'Units', 'Improved',
       'Maintained'])

In [189]:
df.shape

(51013, 23)

In [190]:
# df_2011_2013['crime_counts'] = df_2011_2013.apply(crime_count, axis=1, args=[crimes_coord])

In [191]:
df['distance_km'] = haversine_np(df['lon'],df['lat'],crimes['lon'],crimes['lat'])

In [192]:
df

,lon,lat,ticket_id,violation_street_number,violation_street_name,violation_zip_code,violation_date,judgment_date,payment_date,payment_status,parcel_id,compliance,coordinates,AddCombo,Structure,Condition,Occupancy,Needs_Boarding,Fire,Dumping,Units,Improved,Maintained,distance_km
0,-83.019172,42.339196,18653,2900,JEFFERSON,NaN,2005-01-14,2005-02-23,2005-02-22,PAID IN FULL,11000088,1,"(42.339195641000096, -83.01917209300001)",2900 E JEFFERSON 10,yes,good,occupied,no,no,no,4+ units,not_defined,unknown,11.451846
1,-82.942097,42.388525,18657,15439,MACK,NaN,2005-01-14,2005-02-18,2005-03-09,PAID IN FULL,21001420,0,"(42.3885247860001, -82.942097334)",15433 MACK,yes,fair,unoccupied,no,no,no,1 unit,not_defined,unknown,9.016776
3,-83.045750,42.335121,18661,1310,BROADWAY,NaN,2005-01-14,2005-02-18,2007-06-12,PAID IN FULL,1004004,0,"(42.335120859, -83.0457496719999)",1310 BROADWAY,yes,good,occupied,no,no,no,4+ units,not_defined,unknown,2.993959
5,-83.047308,42.335973,18665,1400,BROADWAY,NaN,2005-01-14,2005-02-18,2008-03-07,PAID IN FULL,1004011,0,"(42.3359729110001, -83.04730827)",1400 BROADWAY,yes,good,occupied,no,no,no,4+ units,not_defined,unknown,11.801459
6,-83.043959,42.335840,18666,426,CLINTON,NaN,2005-01-14,2005-02-18,2009-12-14,PAID IN FULL,1000217,0,"(42.335840128, -83.04395936)",426 CLINTON,yes,good,occupied,no,no,no,4+ units,not_defined,unknown,1.273867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59897,-83.204910,42.414164,537502,16801,RUTHERFORD,48235.0,2020-10-27,2021-01-28,2021-01-28,PAID IN FULL,22056411,1,"(42.414163585, -83.204909868)",16801 RUTHERFORD,yes,good,occupied,no,no,no,1 unit,not_defined,unknown,12.474451
59899,-83.197332,42.346487,538012,7426,WINTHROP,48228.0,2020-11-30,2021-01-27,2021-01-27,PAID IN FULL,22051536,1,"(42.346487472, -83.197331794)",7426 WINTHROP,yes,good,occupied,no,no,no,1 unit,not_defined,unknown,2.955363
59900,-83.128700,42.411274,538444,16174,PRINCETON,48221.0,2020-10-15,2021-02-02,2021-02-02,PAID IN FULL,12011024,1,"(42.411274242, -83.128699952)",16174 PRINCETON,yes,good,occupied,no,no,no,1 unit,not_defined,unknown,5.651995
59901,-83.128700,42.411274,538447,16174,PRINCETON,48221.0,2020-10-15,2021-02-02,2021-02-02,PAID IN FULL,12011024,1,"(42.411274242, -83.128699952)",16174 PRINCETON,yes,good,occupied,no,no,no,1 unit,not_defined,unknown,4.005002
